Importando librerias

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from pprint import pprint
from datetime import datetime
from IPython.display import HTML, display
import folium
import json
import re
import random
import sqlite3

In [ ]:
# Para imprimir toda a informacción de celdas muy largas
pd.set_option('display.max_columns', 500)

# Para imprimir todas las líneas de un dataframe
pd.set_option('display.max_rows', None)

----------------------------------------

extracción datos de meteo

In [4]:
API_KEY = ''
city = "Madrid"
url = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY}&q={city}&days=7&aqi=no&alerts=no&lang=es"

response = requests.get(url)
data = response.json()

for day in data['forecast']['forecastday']:
    date = day['date']
    avg_temp = day['day']['avgtemp_c']
    condition = day['day']['condition']['text']
    will_rain = "rain" in condition.lower()

    print(f"📅 {date}")
    print(f"   🌡️ Temperatura media: {avg_temp}°C")
    print(f"   🌥️ Predicción: {condition}")
    print(f"   ☔ ¿LLoverá? {'Si' if will_rain else 'No'}\n")


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

-------------------------------------------------

webscraping datos de sitio sobre eventos para niños

In [5]:
letras_url = "https://www.planinfantil.es/planes-gratis-con-ninos-en-madrid/"
url_obtenido = requests.get(letras_url)
soup = BeautifulSoup(url_obtenido.text, "html.parser")

# Buscar todos los contenedores de eventos
events_container = soup.find_all('div', class_='wp-block-toolset-blocks-container tb-container')

# Listas para almacenar datos extraídos
events_list_url = []
events_list_title = []
events_list_date = []
events_list_address = []

# Iterar a través de cada contenedor de eventos
for event in events_container:
    try:
        # Obtener el enlace de la imagen / URL del evento
        figure_tag = event.find('figure')
        if figure_tag and figure_tag.find('a'):
            event_url = figure_tag.find('a')['href']
        else:
            continue  # Omitir si no hay URL de evento

        # Obtener título
        title_tag = event.find('h6')
        event_title = title_tag.get_text(strip=True) if title_tag else ""

        # Obtener fecha y dirección
        text_blocks = event.find_all('div', class_='tb-fields-and-text')
        event_date = text_blocks[0].get_text(strip=True) if len(text_blocks) > 0 else ""
        event_address = text_blocks[1].get_text(strip=True) if len(text_blocks) > 1 else ""

        # Añadir a listas
        events_list_url.append(event_url)
        events_list_title.append(event_title)
        events_list_date.append(event_date)
        events_list_address.append(event_address)

    except Exception as e:
        print("Error processing event:", e)

# Crear el DataFrame
df = pd.DataFrame({
    'URL': events_list_url,
    'Title': events_list_title,
    'Date': events_list_date,
    'Place/Price': events_list_address
})

print(df.head())


                                                                                                          URL  \
0         https://www.planinfantil.es/plan/ihe-y-los-sonidos-del-nilo-cuentacuentos-en-el-museo-arqueologico/   
1                       https://www.planinfantil.es/plan/andresillo-y-el-dragon-aventuras-en-el-arqueologico/   
2                     https://www.planinfantil.es/plan/los-sonidos-de-la-antiguedad-en-el-museo-arqueologico/   
3  https://www.planinfantil.es/plan/hipolita-una-reina-de-armas-tomar-cuentacuentos-en-el-museo-arqueologico/   
4                       https://www.planinfantil.es/plan/junia-una-romana-en-apuros-en-el-museo-arqueologico/   

                                                                        Title  \
0          Ihé y los sonidos del Nilo, cuentacuentos en el Museo Arqueológico   
1                        Andresillo y el dragón. Aventuras en el Arqueológico   
2                       Los sonidos de la antigüedad en el Museo Arqueológico 

In [6]:
df.head()

,URL,Title,Date,Place/Price
0,https://www.planinfantil.es/plan/ihe-y-los-sonidos-del-nilo-cuentacuentos-en-el-museo-arqueologico/,"Ihé y los sonidos del Nilo, cuentacuentos en el Museo Arqueológico",Del 11/01/2025 al 07/06/2025,Madrid – Gratis
1,https://www.planinfantil.es/plan/andresillo-y-el-dragon-aventuras-en-el-arqueologico/,Andresillo y el dragón. Aventuras en el Arqueológico,Del 11/01/2025 al 21/06/2025,Madrid – Gratis
2,https://www.planinfantil.es/plan/los-sonidos-de-la-antiguedad-en-el-museo-arqueologico/,Los sonidos de la antigüedad en el Museo Arqueológico,Del 18/01/2025 al 14/06/2025,Madrid – Gratis
3,https://www.planinfantil.es/plan/hipolita-una-reina-de-armas-tomar-cuentacuentos-en-el-museo-arqueologico/,"Hipólita, una reina de armas tomar, cuentacuentos en el Museo Arqueológico",Del 25/01/2025 al 10/05/2025,Madrid – Gratis
4,https://www.planinfantil.es/plan/junia-una-romana-en-apuros-en-el-museo-arqueologico/,"Junia, una romana en apuros en el Museo Arqueológico",Del 15/02/2025 al 24/05/2025,Madrid – Gratis


--------------------------------------------------------------------

recoger datos de sitio eventbrite, filtrado eventos para niños

In [7]:
base_url = "https://www.eventbrite.com/"
target_url = "d/spain/all-events/?subcategories=15003%2C15004%2C15005%2C15006&page=1"


headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,it-IT;q=0.8,it;q=0.7,pt;q=0.6,es;q=0.5",
    "priority": "u=1, i",
    "referer": "https://www.eventbrite.com/d/spain/all-events/?subcategories=15003%2C15004%2C15005%2C15006&page=1",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
    "x-csrftoken": "6a8e093a186e11f0b3df7333c3540c62",
    "x-requested-with": "XMLHttpRequest"
}

In [8]:
resp = requests.get(f'{base_url}/{target_url}', headers=headers)

In [9]:
print(resp.status_code)

200


In [10]:
soup = BeautifulSoup(resp.text, 'html.parser')

In [11]:
soup

{"search_data":{"articles":{},"current_user_id":null,"profile_search":{},"article_search":{},"profiles":{},"events":{"pagination":{"object_count":23,"page_count":2,"page_number":1,"page_size":20,"continuation":"eyJwYWdlIjoyfQ"},"results":[{"image":{"edge_color_set":true,"edge_color":"#5e624b","url":"https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F930362993%2F1055957277713%2F1%2Foriginal.20250108-094813?w=512\u0026auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026rect=0%2C1017%2C7008%2C3504\u0026s=7a92281f824dcd2a8c425567ebf482f8","id":"930362993","crop_mask":{"width":7008,"height":3504,"top_left":{"y":1017,"x":0}},"aspect_ratio":2.0,"focal_point":null,"original":{"url":"https://img.evbuc.com/https%3A%2F%2Fcdn.evbuc.com%2Fimages%2F930362993%2F1055957277713%2F1%2Foriginal.20250108-094813?auto=format%2Ccompress\u0026q=75\u0026sharp=10\u0026s=b174472cd3afab38a581d4e948b3a7aa","width":7008,"height":4672}},"timezone":"Europe/Madrid","id":"1307028560609","tickets_url":"https://

In [12]:
url = "https://www.eventbrite.com/d/spain/all-events/?subcategories=15003%2C15004%2C15005%2C15006&page=1"

headers = {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,it-IT;q=0.8,it;q=0.7,pt;q=0.6,es;q=0.5",
    "priority": "u=1, i",
    "referer": "https://www.eventbrite.com/d/spain/all-events/?subcategories=15003%2C15004%2C15005%2C15006&page=1",
    "sec-ch-ua": '"Google Chrome";v="135", "Not-A.Brand";v="8", "Chromium";v="135"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36",
    "x-csrftoken": "6a8e093a186e11f0b3df7333c3540c62",
    "x-requested-with": "XMLHttpRequest"
}

cookies = {
    "mgrefby": "https://www.google.com/",
    "guest": "identifier=75b1ab71-48c3-4d01-ad8a-782d93c1f5b6&a=13e8&s=3411e4860099bda2b7af7214fd17839fbe2e76413c4fea6258e306e45ffcc717",
    "G": "v=2&i=75b1ab71-48c3-4d01-ad8a-782d93c1f5b6&a=13e8&s=b30ebeb1875f2120a85bd6000eefa890f9f4cf55",
    "csrftoken": "6a8e093a186e11f0b3df7333c3540c62",
    "session": "identifier=822257312e8c43ce8e9bec649fedd725&issuedTs=1744553017&originalTs=1744552296&s=1815426704c353afae38072c44048b429f12aa1d9f323757b7e9fbecd97c17ff"
}

response = requests.get(url, headers=headers, cookies=cookies)

if response.status_code == 200:
    data = response.json()
    search_data = data.get("search_data")
    if search_data and "events" in search_data:
        events = search_data["events"].get("results", [])
        extracted_data = []
        for event in events:
            start_time_str = event.get("start_date") + " " + event.get("start_time") if event.get("start_date") and event.get("start_time") else None
            end_time_str = event.get("end_date") + " " + event.get("end_time") if event.get("end_date") and event.get("end_time") else None
            duration = None
            image_url = event.get("image", {}).get("url")
            venue = event.get("primary_venue", {})
            address = venue.get("address", {})
            longitude = float(address.get("longitude"))
            latitude = float(address.get("latitude"))

            if start_time_str and end_time_str:
                try:
                    start_datetime = datetime.fromisoformat(start_time_str)
                    end_datetime = datetime.fromisoformat(end_time_str)
                    duration = str(end_datetime - start_datetime)
                except ValueError:
                    duration = "Invalid date/time format"

            image_html = f'<img src="{image_url}" width="100" height="100">' if image_url else "No Image"

            item = {
                "name": event.get("name"),
                "url": event.get("url"),
                "image": HTML(image_html),  # Almacenar el objeto HTML
                "start_time": event.get("start_time"),
                "start_date": event.get("start_date"),
                "end_time": event.get("end_time"),
                "end_date": event.get("end_date"),
                "duration": duration,
                "venue_name": event.get("primary_venue", {}).get("name"),
                "venue_address": event.get("primary_venue", {}).get("address", {}).get("localized_address_display"),
                "longitude": longitude,
                "latitude": latitude,
                "summary": event.get("summary"),
                "is_online_event": event.get("is_online_event"),
                "tickets_url": event.get("tickets_url"),
                "tags": [tag.get("display_name") for tag in event.get("tags", [])],
                "event_id": event.get("id"),
                "data_source": "eventbrite"
            }
            extracted_data.append(item)

        # Imprimir los datos extraídos en la salida de Jupyter
        for item in extracted_data:
            print(f"Name: {item['name']}")
            print(f"URL: {item['url']}")
            display(item['image'])  # Utilice display() para renderizar el HTML
            print(f"Start date: {item['start_date']} | End date: {item['end_date']}")
            print(f"Start Time: {item['start_time']}")
            print(f"End Time: {item['end_time']}")
            print(f"Duration: {item['duration']}")
            print("-" * 60)

    else:
        print("Could not find 'events' data in the JSON response.")
else:
    print("Failed to retrieve data:", response.status_code)

Name: Cuánto Cuento!  (de 0 a 6 años)
URL: https://www.eventbrite.es/e/entradas-cuanto-cuento-de-0-a-6-anos-1307028560609


Start date: 2025-04-19 | End date: 2025-04-19
Start Time: 11:30
End Time: 12:00
Duration: 0:30:00
------------------------------------------------------------
Name: Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años
URL: https://www.eventbrite.es/e/entradas-club-futuros-podcasters-taller-de-podcast-para-ninos-de-6-a-12-anos-1297237354849


Start date: 2025-05-08 | End date: 2025-05-08
Start Time: 17:30
End Time: 18:30
Duration: 1:00:00
------------------------------------------------------------
Name: Jugar el arte en familia 3 DE MAYO
URL: https://www.eventbrite.es/e/entradas-jugar-el-arte-en-familia-3-de-mayo-1302039748939


Start date: 2025-05-03 | End date: 2025-05-03
Start Time: 16:30
End Time: 17:30
Duration: 1:00:00
------------------------------------------------------------
Name: Taller FAMILIAR de Lettering  ¡DIBUJAMOS EL ABECEDARIO!
URL: https://www.eventbrite.es/e/taller-familiar-de-lettering-dibujamos-el-abecedario-tickets-1312720154319


Start date: 2025-05-10 | End date: 2025-05-10
Start Time: 10:00
End Time: 12:00
Duration: 2:00:00
------------------------------------------------------------
Name: Investigactiva. Especial Energía y movimiento 18 DE MAYO
URL: https://www.eventbrite.es/e/investigactiva-especial-energia-y-movimiento-18-de-mayo-tickets-1302072707519


...

Start date: 2025-04-27 | End date: 2025-04-27
Start Time: 11:00
End Time: 12:00
Duration: 1:00:00
------------------------------------------------------------
Name: EL REMENDÓN DE LA MUERTE  (MENUTSBARRIS) Títelles (+ 6 años).
URL: https://www.eventbrite.es/e/entradas-el-remendon-de-la-muerte-menutsbarris-titelles-6-anos-1279173094159


Start date: 2025-05-09 | End date: 2025-05-09
Start Time: 18:30
End Time: 19:30
Duration: 1:00:00
------------------------------------------------------------
Name: ÉRASE UNA VEZ UN PATO_Teatro la proa_(MENUTSBARRIS).
URL: https://www.eventbrite.es/e/entradas-erase-una-vez-un-pato-teatro-la-proa-menutsbarris-1277765845039


Start date: 2025-05-09 | End date: 2025-05-09
Start Time: 18:30
End Time: 19:30
Duration: 1:00:00
------------------------------------------------------------
Name: Retratar el alma: retratos de libertad 10 DE MAYO
URL: https://www.eventbrite.es/e/entradas-retratar-el-alma-retratos-de-libertad-10-de-mayo-1300677063109


Start date: 2025-05-10 | End date: 2025-05-10
Start Time: 17:00
End Time: 18:00
Duration: 1:00:00
------------------------------------------------------------
Name: Muchas Caperucitas y algún lobo bueno 11 DE MAYO
URL: https://www.eventbrite.es/e/entradas-muchas-caperucitas-y-algun-lobo-bueno-11-de-mayo-1318017970229


Start date: 2025-05-11 | End date: 2025-05-11
Start Time: 11:00
End Time: 12:00
Duration: 1:00:00
------------------------------------------------------------


In [13]:
eventbrite_df = pd.DataFrame(extracted_data)

In [14]:
eventbrite_df.head()

,name,url,image,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,longitude,latitude,summary,is_online_event,tickets_url,tags,event_id,data_source
0,Cuánto Cuento! (de 0 a 6 años),https://www.eventbrite.es/e/entradas-cuanto-cuento-de-0-a-6-anos-1307028560609,<IPython.core.display.HTML object>,11:30,2025-04-19,12:00,2025-04-19,0:30:00,Vualá - Librería juvenil y infantil,"17 Calle de Santa Feliciana Calle Santa Feliciana 17, 28010 Madrid, España, 28010 Madrid",-3.698552,40.433543,"¡Ven y disfruta de una mañana llena de historias, risas y diversión en nuestros cuenta cuentos, junto a Factoría Teatro!",False,https://www.eventbrite.com/checkout-external?eid=1307028560609,"[Children & Youth , Family & Education, Concert or Performance, Children, Kids, Event, Storytelling, Familia, Evento, Madrid, Cuentos, Cuentacuentos, Libreria]",1307028560609,eventbrite
1,Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años,https://www.eventbrite.es/e/entradas-club-futuros-podcasters-taller-de-podcast-para-ninos-de-6-a-12-anos-1297237354849,<IPython.core.display.HTML object>,17:30,2025-05-08,18:30,2025-05-08,1:00:00,"C. de Ramón y Cajal, 4","4 Calle de Ramón y Cajal, 28801 Alcalá de Henares",-3.365515,40.482542,Talleres de podcast mensuales para niños. Nos reunimos el primer jueves de cada mes. Ven a probar un taller o apúntate a todos hasta julio.,False,https://www.eventbrite.com/checkout-external?eid=1297237354849,"[Children & Youth , Family & Education, Class, Training, or Workshop, Radio, Audio, Taller, Podcast, Educacion, Comunicación, Curso, Ocio, Locutores, Microfono]",1297237354849,eventbrite
2,Jugar el arte en familia 3 DE MAYO,https://www.eventbrite.es/e/entradas-jugar-el-arte-en-familia-3-de-mayo-1302039748939,<IPython.core.display.HTML object>,16:30,2025-05-03,17:30,2025-05-03,1:00:00,Casa del Lector,"14 Paseo de la Chopera, 28045 Madrid",-3.698443,40.392663,Explorando y creando juntos,False,https://www.eventbrite.com/checkout-external?eid=1302039748939,"[Children & Youth , Family & Education, Class, Training, or Workshop]",1302039748939,eventbrite
3,Taller FAMILIAR de Lettering ¡DIBUJAMOS EL ABECEDARIO!,https://www.eventbrite.es/e/taller-familiar-de-lettering-dibujamos-el-abecedario-tickets-1312720154319,<IPython.core.display.HTML object>,10:00,2025-05-10,12:00,2025-05-10,2:00:00,Tienda Cuadernos RUBIO,"9 Carrer de Sorní #bajo derecha, 46004 València",-0.368432,39.470320,"Ven y aprende a dibujar letras de forma creativa en familia en nuestro taller de lettering, ¡te esperamos en la tienda RUBIO!",False,https://www.eventbrite.com/checkout-external?eid=1312720154319,"[Parenting, Family & Education, Class, Training, or Workshop, Workshop, Lettering, Familiar, abecedario, dibujamos]",1312720154319,eventbrite
4,Investigactiva. Especial Energía y movimiento 18 DE MAYO,https://www.eventbrite.es/e/investigactiva-especial-energia-y-movimiento-18-de-mayo-tickets-1302072707519,<IPython.core.display.HTML object>,11:30,2025-05-18,12:25,2025-05-18,0:55:00,Casa del Lector,"14 Paseo de la Chopera, 28045 Madrid",-3.698443,40.392663,Espacio de experimentación científica para niñas y niños de 2 a 8 años,False,https://www.eventbrite.com/checkout-external?eid=1302072707519,"[Children & Youth , Family & Education, Class, Training, or Workshop, Conferencias, Evento, Actividades, Octubre, investigactiva]",1302072707519,eventbrite


In [15]:
# Generando el mapa a partir de las coordenadas

# Crea un mapa base centrado en España
spain_latitude = 40.4637
spain_longitude = -3.7492
event_map = folium.Map(location=[spain_latitude, spain_longitude], zoom_start=7)

# Iterar a través del DataFrame y agregar marcadores
for _, row in eventbrite_df.iterrows():
    popup_html = f"""
    <strong>{row['name']}</strong><br>
    📍 <em>{row['venue_name']}</em><br>
    🗓 {row['start_date']} {row['start_time']} - {row['end_time']}<br>
    <a href="{row['url']}" target="_blank">Ver evento</a>
    """
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=row['name']
    ).add_to(event_map)

# Guardar en HTML:
event_map.save("event_map.html")

# o mostrar en el notebook:
event_map


--------------------------------------------------------

api datos.madrid.es

In [16]:
# descargando los datos de Agenda de actividades y eventos de la API del Portal de datos abiertos del Ayuntamiento de Madrid

url = "https://datos.madrid.es/egob/catalogo/300107-0-agenda-actividades-eventos.json"

try:
    response = requests.get(url)
    response.raise_for_status()
    raw_text = response.text
    cleaned_text = re.sub(r"\\(?![nrtbf\"\\/])", "", raw_text)
    data = json.loads(cleaned_text)

    if isinstance(data, dict):
        print(f"Top-level dictionary keys: {data.keys()}")
        if '@graph' in data and isinstance(data['@graph'], list):
            df = pd.DataFrame(data['@graph'])
            print(df.head())
        else:
            print("The list of events was not found within the '@graph' key.")
    else:
        print("Parsed JSON is not a dictionary as expected.")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Top-level dictionary keys: dict_keys(['@context', '@graph'])
                                                                                                                      @id  \
0                     https://datos.madrid.es/egob/catalogo/tipo/evento/12733698-150-anos-nacimiento-antonio-machado.json   
1  https://datos.madrid.es/egob/catalogo/tipo/evento/12735098-150-anos-nacimiento-antonio-machado-teatro-ojos-poeta-.json   
2                                https://datos.madrid.es/egob/catalogo/tipo/evento/12734393-2-encuentro-estuve-alli-.json   
3                         https://datos.madrid.es/egob/catalogo/tipo/evento/12750408-6-sesion-poetas-americas-madrid.json   
4                https://datos.madrid.es/egob/catalogo/tipo/evento/12747998-75-aniversario-adhesion-hortaleza-madrid.json   

                                                                  @type  \
0    https://datos.madrid.es/egob/kos/actividades/ConferenciasColoquios   
1  https://datos.madrid.es/egob/kos/ac

In [17]:
# Descubriendo los tipos de público registrados en el dataset para luego filtrar por las categorias que nos interesan
df['audience'].unique()

array([nan, 'Familias', 'Niños', 'Jovenes,Niños', 'Jovenes', 'Mayores',
       'Mujeres', 'Niños,Familias', 'Mujeres,Mayores',
       'Jovenes,Niños,Familias', 'EmpresariosYComerciantes',
       'Discapacitados', 'Familias,Mayores',
       'Jovenes,EstudianteseInvestigadores'], dtype=object)

In [18]:
# Haciendo el filtrado del dataset
filtered_df = df[df['audience'].isin(['Familias', 'Niños', 'Jovenes,Niños', 'Niños,Familias', 'Jovenes,Niños,Familias'])]

In [19]:
filtered_df.head(1)

,@id,@type,id,title,description,free,price,dtstart,dtend,time,...,uid,link,event-location,references,relation,address,location,organization,audience,recurrence
1,https://datos.madrid.es/egob/catalogo/tipo/evento/12735098-150-anos-nacimiento-antonio-machado-teatro-ojos-poeta-.json,https://datos.madrid.es/egob/kos/actividades/ComemoracionesHomenajes,12735098,150 AÑOS NACIMIENTO ANTONIO MACHADO. Teatro: 'Con ojos de poeta',"Dramaturgia: Jesús Novo- Poesías: Antonio Machado Con:- Sonsoles Novo (Dirección artística e intérprete). Álida Jiménez (Dirección Musical e intérprete: voz y guitarra). En la casa de Petrita, que fue alumna de Don Antonio Machado, cada vez que se retira una tela aparece un objeto relacionado con la vida y la obra de Machado. Edad recomendada: público adulto y apto a partir de 8 años.",1,,2025-04-25 18:30:00.0,2025-04-25 23:59:00.0,18:30,...,12735098,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=39af8b56808d5910VgnVCM2000001f4a900aRCRD,Centro Sociocultural Montecarmelo (Fuencarral - El Pardo),{'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=a2fea399e519c810VgnVCM1000001d4a900aRCRD'},{'@id': 'https://datos.madrid.es/egob/catalogo/tipo/entidadesyorganismos/5514552-centro-sociocultural-montecarmelo-fuencarral-pardo-.json'},"{'district': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Fuencarral-ElPardo'}, 'area': {'@id': 'https://datos.madrid.es/egob/kos/Provincia/Madrid/Municipio/Madrid/Distrito/Fuencarral-ElPardo/Barrio/ElGoloso', 'locality': 'MADRID', 'postal-code': '28049', 'street-address': 'Carretera M-607 Km 13 L-10 '}}","{'latitude': 40.515712291624084, 'longitude': -3.6860532895354448}","{'organization-name': 'Centro Sociocultural Montecarmelo (Fuencarral - El Pardo)', 'accesibility': '1'}",Familias,NaN


In [20]:
# Filtrando el dataset por valores nulos en las columnas que hacen referencia a la ubicación del evento
filtered_df[filtered_df['address'].isna() & filtered_df['location'].isna()].head()

,@id,@type,id,title,description,free,price,dtstart,dtend,time,...,uid,link,event-location,references,relation,address,location,organization,audience,recurrence
117,https://datos.madrid.es/egob/catalogo/tipo/evento/12702034-campamento-verano-2025-puente-vallecas.json,https://datos.madrid.es/egob/kos/actividades/ActividadesCalleArteUrbano,12702034,Campamento de verano 2025 en Puente de Vallecas,,1,,2025-06-23 00:00:00.0,2025-09-05 23:59:00.0,,...,12702034,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=5c4cb1e2d1745910VgnVCM1000001d4a900aRCRD,,{'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=bc9ab1e2d1745910VgnVCM1000001d4a900aRCRD'},NaN,NaN,NaN,NaN,"Niños,Familias","{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': 'WEEKLY', 'interval': 1}"
118,https://datos.madrid.es/egob/catalogo/tipo/evento/12752479-campamento-verano-2025-distrito-villaverde.json,https://datos.madrid.es/egob/kos/actividades/Campamentos,12752479,Campamento de Verano 2025 - Distrito de Villaverde,,1,,2025-06-23 00:00:00.0,2025-07-31 23:59:00.0,,...,12752479,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=50838bbda3026910VgnVCM2000001f4a900aRCRD,,{'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=e3cf8bbda3026910VgnVCM2000001f4a900aRCRD'},NaN,NaN,NaN,NaN,Niños,"{'days': 'MO,TU,WE,TH,FR', 'frequency': 'WEEKLY', 'interval': 1}"
123,https://datos.madrid.es/egob/catalogo/tipo/evento/12683216-campamento-urbano-primavera-usera.json,https://datos.madrid.es/egob/kos/actividades/Campamentos,12683216,Campamento urbano de primavera en Usera,,1,,2025-04-11 00:00:00.0,2025-04-21 23:59:00.0,,...,12683216,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=9441898ec60f4910VgnVCM2000001f4a900aRCRD,,{'@id': 'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=456e1a0b100f4910VgnVCM2000001f4a900aRCRD'},NaN,NaN,NaN,NaN,Niños,"{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': 'WEEKLY', 'interval': 1}"
126,https://datos.madrid.es/egob/catalogo/tipo/evento/12753682-campamento-urbano-verano-usera.json,https://datos.madrid.es/egob/kos/actividades/Campamentos,12753682,Campamento urbano de verano en Usera,,1,,2025-07-01 00:00:00.0,2025-09-05 23:59:00.0,,...,12753682,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=514e2e4d2e136910VgnVCM2000001f4a900aRCRD,,{'@id': ''},NaN,NaN,NaN,NaN,Niños,"{'days': 'MO,TU,WE,TH,FR,SA,SU', 'frequency': 'WEEKLY', 'interval': 1}"
135,https://datos.madrid.es/egob/catalogo/tipo/evento/12741335-caracterizacion-actividades-infantiles.json,https://datos.madrid.es/egob/kos/actividades/RecitalesPresentacionesActosLiterarios,12741335,Caracterización y actividades infantiles,Caracterización y actividades infantiles entorno al Siglo de Oro. Lugar: Paseo del Prado 36 (CaixaFoum).,1,,2025-04-24 17:00:00.0,2025-04-24 23:59:00.0,17:00,...,12741335,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=fa285b230f5f5910VgnVCM2000001f4a900aRCRD,,{'@id': ''},NaN,NaN,NaN,NaN,"Niños,Familias",NaN


In [21]:
extracted_data = []
for index, row in filtered_df.iterrows():
    extracted_event = {
        "name": row["title"],
        "url": row["link"],
        "image": None,
        "start_time": row["time"],
        "start_date": pd.to_datetime(row["dtstart"]).date() if pd.notna(row["dtstart"]) else None,
        "end_time": None,
        "end_date": pd.to_datetime(row["dtend"]).date() if pd.notna(row["dtend"]) else None,
        "duration": None,
        "venue_name": row["event-location"],
        "venue_address": row["address"].get("area", {}).get("street-address")
                         if isinstance(row["address"], dict) else None,
        "longitude": row["location"].get("longitude")
                         if isinstance(row["location"], dict) else None, # Comprueba si la ubicación es un diccionario
        "latitude": row["location"].get("latitude")
                        if isinstance(row["location"], dict) else None, # Comprueba si la ubicación es un diccionario
        "summary": row["description"],
        "is_online_event": None,
        "tickets_url": row["link"],
        "tags": [row["audience"]] if pd.notna(row["audience"]) else [],
        "event_id": row["uid"],
        "data_source": "ayuntamiento madrid"
    }
    extracted_data.append(extracted_event)

ayuntamiento_madrid_df = pd.DataFrame(extracted_data)
print(ayuntamiento_madrid_df.head())
print(ayuntamiento_madrid_df.info())

                                                               name  \
0  150 AÑOS NACIMIENTO ANTONIO MACHADO. Teatro: 'Con ojos de poeta'   
1                                                   A la mar salada   
2                                               A taste of the moon   
3                                               A taste of the moon   
4                                               A taste of the moon   

                                                                                                                                                url  \
0  http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=39af8b56808d5910VgnVCM2000001f4a900aRCRD   
1  http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=366f468e9b336910VgnVCM1000001d4a900aRCRD   
2  http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=edd0528353436

In [22]:
ayuntamiento_madrid_df.head()

,name,url,image,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,longitude,latitude,summary,is_online_event,tickets_url,tags,event_id,data_source
0,150 AÑOS NACIMIENTO ANTONIO MACHADO. Teatro: 'Con ojos de poeta',http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=39af8b56808d5910VgnVCM2000001f4a900aRCRD,None,18:30,2025-04-25,None,2025-04-25,None,Centro Sociocultural Montecarmelo (Fuencarral - El Pardo),Carretera M-607 Km 13 L-10,-3.686053,40.515712,"Dramaturgia: Jesús Novo- Poesías: Antonio Machado Con:- Sonsoles Novo (Dirección artística e intérprete). Álida Jiménez (Dirección Musical e intérprete: voz y guitarra). En la casa de Petrita, que fue alumna de Don Antonio Machado, cada vez que se retira una tela aparece un objeto relacionado con la vida y la obra de Machado. Edad recomendada: público adulto y apto a partir de 8 años.",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=39af8b56808d5910VgnVCM2000001f4a900aRCRD,[Familias],12735098,ayuntamiento madrid
1,A la mar salada,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=366f468e9b336910VgnVCM1000001d4a900aRCRD,None,18:00,2025-05-09,None,2025-05-09,None,Biblioteca Pública Municipal Ciudad Lineal (Ciudad Lineal),CALLE HERMANOS GARCIA NOBLEJAS 14,-3.637292,40.436447,"Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. Recuerda que las niñas y niños deberán ir acompañados por una persona adulta. Para asistir es necesario realizar inscripción previa a través de este enlace . Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de Margalida Alberti.",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=366f468e9b336910VgnVCM1000001d4a900aRCRD,[Niños],12753990,ayuntamiento madrid
2,A taste of the moon,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=edd0528353436910VgnVCM2000001f4a900aRCRD,None,17:30,2025-05-22,None,2025-05-22,None,Biblioteca Pública Municipal Pablo Neruda (Ciudad Lineal),CALLE ASCAO 4,-3.641690,40.429411,"Vamos a leer en inglés y a divertirnos en la biblioteca. Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. Para asistir es necesaria la inscripción a través de este enlace . La biblioteca se pondrá en contacto contigo para confirmar tu asistencia. Recuerda que los niños y niñas deberán ir acompañados por una persona adulta. Te esperamos en la biblioteca para pasar un buen rato lleno de fantasía, de la mano de Mamen Storyteller.",None,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=edd0528353436910VgnVCM2000001f4a900aRCRD,[Niños],12754420,ayuntamiento madrid
3,A taste of the moon,http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=7911528353436910VgnVCM2000001f4a900aRCRD,None,17:30,2025-05-29,None,2025-05-29,None,Biblioteca Pública Municipal Manuel Vázquez Montalbán (Tetuán),CALLE FRANCOS RODRIGUEZ 67,-3.710540,40.457495,"Vamos a leer en inglés y a divertirnos en la biblioteca. Duendes, tesoros, aventuras, misterios... Las bibliotecas están llenas de historias que cuenteros y cuenteras nos acercan a través de su creatividad y su magia. Este cuento está pensado para niñas y niños a partir de 2 años. La entrada es libre hasta completar aforo. Recuerda que los niños y niñas deberán ir acompañados por una persona adulta. Te esperamos en la biblioteca para pasar un buen rato lleno de fantasí

In [23]:
# Generando el mapa a partir de las coordenadas

# Filtrar filas donde la latitud o longitud es NaN
valid_events_df = ayuntamiento_madrid_df.dropna(subset=['latitude', 'longitude'])

if not valid_events_df.empty:
    # Crea un mapa base centrado en España
    spain_latitude = 40.4637
    spain_longitude = -3.7492
    event_map = folium.Map(location=[spain_latitude, spain_longitude], zoom_start=7)

    # Iterar a través del DataFrame filtrado y agregar marcadores
    for _, row in valid_events_df.iterrows():
        popup_html = f"""
        <strong>{row['name']}</strong><br>
        📍 <em>{row['venue_name']}</em><br>
        🗓 {row['start_date']} {row['start_time']} - {row['end_time']}<br>
        <a href="{row['url']}" target="_blank">Ver evento</a>
        """
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=row['name']
        ).add_to(event_map)

    # Guardar como HTML:
    event_map.save("event_map.html")
    # o mostrar en el notebook:
    event_map
else:
    print("No events with valid latitude and longitude to display on the map.")

In [24]:
event_map

---------------------------------------------

Concatenando los dataframes

In [ ]:
# Creando una lista con los datasets existentes
dataframes_list = [eventbrite_df, ayuntamiento_madrid_df]

In [26]:
# Concatenando los dataframes
combined_df = pd.concat(dataframes_list, ignore_index=True)

In [27]:
combined_df = combined_df.drop(columns='image')

In [28]:
combined_df.head()

,name,url,start_time,start_date,end_time,end_date,duration,venue_name,venue_address,longitude,latitude,summary,is_online_event,tickets_url,tags,event_id,data_source
0,Cuánto Cuento! (de 0 a 6 años),https://www.eventbrite.es/e/entradas-cuanto-cuento-de-0-a-6-anos-1307028560609,11:30,2025-04-19,12:00,2025-04-19,0:30:00,Vualá - Librería juvenil y infantil,"17 Calle de Santa Feliciana Calle Santa Feliciana 17, 28010 Madrid, España, 28010 Madrid",-3.698552,40.433543,"¡Ven y disfruta de una mañana llena de historias, risas y diversión en nuestros cuenta cuentos, junto a Factoría Teatro!",False,https://www.eventbrite.com/checkout-external?eid=1307028560609,"[Children & Youth , Family & Education, Concert or Performance, Children, Kids, Event, Storytelling, Familia, Evento, Madrid, Cuentos, Cuentacuentos, Libreria]",1307028560609,eventbrite
1,Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años,https://www.eventbrite.es/e/entradas-club-futuros-podcasters-taller-de-podcast-para-ninos-de-6-a-12-anos-1297237354849,17:30,2025-05-08,18:30,2025-05-08,1:00:00,"C. de Ramón y Cajal, 4","4 Calle de Ramón y Cajal, 28801 Alcalá de Henares",-3.365515,40.482542,Talleres de podcast mensuales para niños. Nos reunimos el primer jueves de cada mes. Ven a probar un taller o apúntate a todos hasta julio.,False,https://www.eventbrite.com/checkout-external?eid=1297237354849,"[Children & Youth , Family & Education, Class, Training, or Workshop, Radio, Audio, Taller, Podcast, Educacion, Comunicación, Curso, Ocio, Locutores, Microfono]",1297237354849,eventbrite
2,Jugar el arte en familia 3 DE MAYO,https://www.eventbrite.es/e/entradas-jugar-el-arte-en-familia-3-de-mayo-1302039748939,16:30,2025-05-03,17:30,2025-05-03,1:00:00,Casa del Lector,"14 Paseo de la Chopera, 28045 Madrid",-3.698443,40.392663,Explorando y creando juntos,False,https://www.eventbrite.com/checkout-external?eid=1302039748939,"[Children & Youth , Family & Education, Class, Training, or Workshop]",1302039748939,eventbrite
3,Taller FAMILIAR de Lettering ¡DIBUJAMOS EL ABECEDARIO!,https://www.eventbrite.es/e/taller-familiar-de-lettering-dibujamos-el-abecedario-tickets-1312720154319,10:00,2025-05-10,12:00,2025-05-10,2:00:00,Tienda Cuadernos RUBIO,"9 Carrer de Sorní #bajo derecha, 46004 València",-0.368432,39.470320,"Ven y aprende a dibujar letras de forma creativa en familia en nuestro taller de lettering, ¡te esperamos en la tienda RUBIO!",False,https://www.eventbrite.com/checkout-external?eid=1312720154319,"[Parenting, Family & Education, Class, Training, or Workshop, Workshop, Lettering, Familiar, abecedario, dibujamos]",1312720154319,eventbrite
4,Investigactiva. Especial Energía y movimiento 18 DE MAYO,https://www.eventbrite.es/e/investigactiva-especial-energia-y-movimiento-18-de-mayo-tickets-1302072707519,11:30,2025-05-18,12:25,2025-05-18,0:55:00,Casa del Lector,"14 Paseo de la Chopera, 28045 Madrid",-3.698443,40.392663,Espacio de experimentación científica para niñas y niños de 2 a 8 años,False,https://www.eventbrite.com/checkout-external?eid=1302072707519,"[Children & Youth , Family & Education, Class, Training, or Workshop, Conferencias, Evento, Actividades, Octubre, investigactiva]",1302072707519,eventbrite


generando un nuevo mapa con ele dataset combinado, distinguiendo cada fonte de datos por color

In [29]:
# Generando el mapa a partir de las coordenadas con colores únicos por fuente

# Obtener las fuentes de datos únicas
unique_sources = combined_df['data_source'].unique()
num_sources = len(unique_sources)

# Define una lista de colores
available_colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'beige', 'pink', 'black'
                    ]

# Mezcla los colores para asegurar una asignación algo aleatoria
random.shuffle(available_colors)
num_available_colors = len(available_colors)

# Crea un diccionario para asignar cada fuente única a un color
source_colors = {}
for i, source in enumerate(unique_sources):
    if i < num_available_colors:
        source_colors[source] = available_colors[i]
    else:
        # Manejar casos donde hay más fuentes que colores
        # Quizás quieras implementar una generación de color más sofisticada aquí
        # Para simplificar, volveremos a recorrer los colores, pero con una advertencia
        source_colors[source] = available_colors[i % num_available_colors]
        print(f"Warning: More data sources ({num_sources}) than available unique colors ({num_available_colors}). Colors may repeat.")

# Filtrar filas sin latitud o longitud
valid_events_df = combined_df.dropna(subset=['latitude', 'longitude', 'data_source'])

if not valid_events_df.empty:
    spain_latitude = 40.4637
    spain_longitude = -3.7492

    # Crear un mapa base centrado en España
    event_map = folium.Map(location=[spain_latitude, spain_longitude], zoom_start=7)

    # Iterar a través del DataFrame filtrado y agregar marcadores con color según data_source
    for _, row in valid_events_df.iterrows():
        data_source = row['data_source']
        marker_color = source_colors.get(data_source, 'gray')  # Predeterminado en gris si no se encuentra la fuente

        popup_html = f"""
        <strong>{row['name']}</strong><br>
        Fuente: <em>{data_source}</em><br>
        📍 <em>{row['venue_name']}</em><br>
        🗓 {row['start_date']} {row['start_time']} - {row['end_time']}<br>
        <a href="{row['url']}" target="_blank">Ver evento</a>
        """
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{row['name']} ({data_source})",
            icon=folium.Icon(color=marker_color)
        ).add_to(event_map)

    # Save como HTML:
    # event_map.save("event_map_unique_colored.html")
    
    # Para mostrar el mapa en el notebook:
    event_map
else:
    print("No events with valid latitude, longitude, or data_source to display on the map.")

In [30]:
event_map

In [31]:
# Crear la base de datos

DATABASE_FILE = 'events.db'
TABLE_NAME = 'all_events'
UNIQUE_ID_COLUMN = 'event_id'
LIST_COLUMNS_TO_CONVERT = ['tags']  # Convertir la columna tags de lista para formato string, la base de datos no accepta inserción del formato lista de python

def update_event_database(current_events_df):
    conn = sqlite3.connect(DATABASE_FILE)
    cursor = conn.cursor()

    # Obtener la lista de columnas del DataFrame, excluyendo la columna de ID única
    columns_list = [col for col in current_events_df.columns if col != UNIQUE_ID_COLUMN]
    columns_str = ', '.join(columns_list)
    placeholders = ', '.join(['?'] * len(columns_list))

    # Crear la tabla
    create_table_sql = f"CREATE TABLE IF NOT EXISTS {TABLE_NAME} ({UNIQUE_ID_COLUMN} TEXT PRIMARY KEY, {columns_str})"
    cursor.execute(create_table_sql)
    conn.commit()

    # Obtener datos unicos existentes de la base de datos
    cursor.execute(f"SELECT {UNIQUE_ID_COLUMN} FROM {TABLE_NAME}")
    existing_ids = {row[0] for row in cursor.fetchall()}

    new_events_to_insert = []
    for index, row in current_events_df.iterrows():
        unique_id = row[UNIQUE_ID_COLUMN]
        if unique_id not in existing_ids:
            data_to_insert = [unique_id]
            for col in columns_list:
                value = row[col]
                # Convertir objetos HTML a su representación de cadena
                if isinstance(value, HTML):
                    data_to_insert.append(str(value))
                # Convertir listas a una representación de string (por ejemplo, separadas por comas)
                elif isinstance(value, list):
                    data_to_insert.append(','.join(map(str, value)))
                else:
                    data_to_insert.append(value)
            new_events_to_insert.append(tuple(data_to_insert))  # Utilice tupla para ejecutar varios

    if new_events_to_insert:
        insert_placeholders = ', '.join(['?'] * (len(columns_list) + 1))
        insert_sql = f"INSERT INTO {TABLE_NAME} VALUES ({insert_placeholders})"
        cursor.executemany(insert_sql, new_events_to_insert)
        conn.commit()
        print(f"Inserted {len(new_events_to_insert)} new events into the database.")
    else:
        print("No new events found.")

    conn.close()

print("First few rows of combined_df:")
print(combined_df.head())

update_event_database(combined_df)

# Para cargar todos los eventos (incluidos los históricos) de la base de datos:
def load_all_events_from_db():
    conn = sqlite3.connect(DATABASE_FILE)
    all_events_df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", conn)
    conn.close()
    return all_events_df

all_events_df = load_all_events_from_db()
print("\nAll events loaded from the database:")
print(all_events_df.head())

First few rows of combined_df:
                                                                    name  \
0                                        Cuánto Cuento!  (de 0 a 6 años)   
1  Club Futuros Podcasters - Taller de podcast para niños de 6 a 12 años   
2                                     Jugar el arte en familia 3 DE MAYO   
3                Taller FAMILIAR de Lettering  ¡DIBUJAMOS EL ABECEDARIO!   
4               Investigactiva. Especial Energía y movimiento 18 DE MAYO   

                                                                                                                      url  \
0                                          https://www.eventbrite.es/e/entradas-cuanto-cuento-de-0-a-6-anos-1307028560609   
1  https://www.eventbrite.es/e/entradas-club-futuros-podcasters-taller-de-podcast-para-ninos-de-6-a-12-anos-1297237354849   
2                                   https://www.eventbrite.es/e/entradas-jugar-el-arte-en-familia-3-de-mayo-1302039748939   
3           

In [32]:
pd.read_sql_query("SELECT * FROM all_events", sqlite3.connect('events.db'))
pd.read_sql_query("select count(*) from all_events", sqlite3.connect('events.db'))

,count(*)
0,304
